In [2]:
library(tidyverse)
library(repr)
library(infer)
library(cowplot)
library(broom)
library(GGally)
library(AER)
library(janitor)
library(scales)
library(latex2exp)
library(tidymodels)
library(gridExtra)
library(faraway)
library(mltools)
library(leaps)
library(glmnet)
library(cowplot)
library(tibble)

Create an electronic report with a maximum of 2000 words (excluding citations) using Jupyter. The report should include the posed question, conducted analysis, and derived conclusion. Only one team member needs to submit this report. It is not required to include all tasks completed by each group member in their individual assignments. Make sure to reach a consensus among all team members on the final content of the report. If needed, consult your TA and Instructor for further guidance.

## Title

### Introduction

Start with relevant background information on the topic to prepare those unfamiliar for the rest of your proposal.

Formulate one or two questions for investigation and detail the dataset that will be utilized to address these questions.

Additionally, align your question/objectives with the existing literature. To contextualize your study, include a minimum of two scientific publications (these should be listed in the References section).

Our question will be the following:

*Which explanatory variables best predict stars on a GitHub repository?*

Surveys confirm that stars are viewed by pratictioners as the most useful measure of a repository's popularity (Borges & Valente, 2018). As GitHub becomes more and more prevalent among developers for use in version control, it can be interesting to see what variables best predict popularity. Recent attempts have been made to solve this problem, such as Moid et. al. with a Random Forest regressor, but no research exists on using mulitple linear regression and LASSO feature selection on this problem (2021). We will explore this below./

### Methods and Results

In this section, you will include:

a) “Exploratory Data Analysis (EDA)”

Demonstrate that the dataset can be read into R.
Clean and wrangle your data into a tidy format.
Plot the relevant raw data, tailoring your plot to address your question.
Make sure to explore the association of the explanatory variables with the response.
Any summary tables that are relevant to your analysis.
Be sure not to print output that takes up a lot of screen space.
Your EDA must be comprehensive with high quality plots.

Describe in written English the methods you used to perform your analysis from beginning to end, and narrate the code that does the analysis.
If included, describe the “Feature Selection” process and how and why you choose the covariates of your final model.
Make sure to interpret/explain the results you obtain. It’s not enough to just say, “I fitted a linear model with these covariates, and my R-square is 0.87”.
If inference is the aim of your project, a detailed interpretation of your fitted model is required, as well as a discussion of relevant quantities (e.g., are the coefficients significant? How does the model fit the data)?
A careful model assessment must be conducted.
If prediction is the project's aim, describe the test data used or how it was created.
Ensure your tables and/or figures are labelled with a figure/table number.

### Discussion

In this section, you’ll interpret the results you obtained in the previous section with respect to the main question/goal of your project.

Summarize what you found and the implications/impact of your findings.
If relevant, discuss whether your results were what you expected to find.
Discuss how your model could be improved;
Discuss future questions/research this study could lead to.

### References

At least two citations of literature relevant to the project. The citation format is your choice – just be consistent. Make sure to cite the source of your data as well.

Borges, Hudson, and Marco Tulio Valente. “What’s in a GitHub Star? Understanding Repository Starring Practices in a Social Coding Platform.” Journal of Systems and Software, vol. 146, Dec. 2018, pp. 112–129, www.sciencedirect.com/science/article/pii/S0164121218301961, https://doi.org/10.1016/j.jss.2018.09.016. Accessed 4 Nov. 2019.

Moid, Mohammed Abdul, et al. “Predicting Stars on Open-Source GitHub Projects.” 2022 Smart Technologies, Communication and Robotics (STCR), 9 Oct. 2021, ieeexplore.ieee.org/document/9588891, https://doi.org/10.1109/stcr51658.2021.9588891. Accessed 5 Dec. 2024.